In [ ]:
import pandas as pd
import altair as alt

df=pd.read_csv("/content/drive/MyDrive/concierto_database_limpia_2.csv", delimiter=';')


In [ ]:
df

,ID,Concierto,Recinto,Productora/organizadora,Tiquetera,Fecha,Numero_asistentes,Tipo_ artista,Artista,Rango_precios,Opcion_descuento_entradas,Preventa,Cancelado,Cambio_recinto,Genero_musical
0,1,X The Future is Unknown,Club Hipico,Street Machine,PuntoTicket,28-09-2024,Sin datos,Internacional,Adriatique,39.000 - 99.000,No,1,0,0,Electronica
1,1,X The Future is Unknown,Club Hipico,Street Machine,PuntoTicket,28-09-2024,Sin datos,Internacional,Mau P,39.000 - 99.000,No,1,0,0,Electronica
2,1,X The Future is Unknown,Club Hipico,Street Machine,PuntoTicket,28-09-2024,Sin datos,Internacional,Layla Benítez,39.000 - 99.000,No,1,0,0,Electronica
3,1,X The Future is Unknown,Club Hipico,Street Machine,PuntoTicket,28-09-2024,Sin datos,Internacional,Undercatt y Vomee,39.000 - 99.000,No,1,0,0,Electronica
4,1,X The Future is Unknown,Club Hipico,Street Machine,PuntoTicket,28-09-2024,Sin datos,Nacional,Camila Govorcín,39.000 - 99.000,No,1,0,0,Electronica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,221,Celebrando 60 años de musica: The Ska-talites,Teatro Caupolican,FFBOOKING,PuntoTicket,08-12-2024,Sin datos,Nacional,Chico Trujillo,20.000 -,No,0,0,0,Ska
245,221,Beyond The World Latin America Tour 2024,Teatro Caupolican,Atenea Producciones,PuntoTicket,10-12-2024,Sin datos,Internacional,Tokio Hotel,42.000 - 46.000,No,0,0,0,Rock
246,222,Jinjer Latin America Tour 2024,Teatro Caupolican,Napalm Events,PuntoTicket,12-12-2024,Sin datos,Internacional,Jinjer,38.000 - 85.000,No,1,0,0,Metal
247,223,Jinjer Latin America Tour 2025,Teatro Caupolican,Napalm Events,PuntoTicket,12-12-2024,Sin datos,Internacional,Heaven Shall Burn,38.000 - 85.000,No,1,0,0,Metal


In [ ]:
df['Fecha']=pd.to_datetime(df['Fecha'], dayfirst=True)

df=df.sort_values(by=['Fecha'])

In [ ]:
import numpy as np
import pandas as pd
from urllib import request
import json
import altair as alt

source = pd.read_csv('/content/drive/MyDrive/concierto_database_limpia_2.csv', delimiter=';')

# Asegurarse de que la columna es de tipo fecha
source['Fecha'] = pd.to_datetime(source['Fecha'], format='%d-%m-%Y')
source['Mes'] = source['Fecha'].dt.month

# Añadir una columna Y aleatoria para separar puntos verticalmente
np.random.seed(42)  # para reproducibilidad
source['y_random'] = np.random.uniform(0, 1, size=len(source))

# Recupera el archivo en español que está online acá: https://github.com/d3/d3-time-format/tree/main/locale
#  y activa el formato español & timeFormat locales.

with request.urlopen('https://raw.githubusercontent.com/d3/d3-format/master/locale/es-MX.json') as f:
  de_format = json.load(f)

with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-MX.json') as f:
  de_time_format = json.load(f)

# setea las opciones de alt (nuestro objeto altair) para que todos los gráficos que generen tengan las cosas en español
alt.renderers.set_embed_options(formatLocale=de_format, timeFormatLocale=de_time_format)
# Leer y preparar datos

source = pd.read_csv('/content/drive/MyDrive/concierto_database_limpia_2.csv', delimiter=';')

source['Fecha'] = pd.to_datetime(source['Fecha'], format='%d-%m-%Y', errors='coerce')

# FILTRAR: eliminar conciertos cancelados
source = source[source['Cancelado'] != 1]

np.random.seed(42)
source['y_random'] = np.random.uniform(0, 1, size=len(source))

# Diccionario para traducir meses
meses_es = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
    5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
    9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}

# Crear nueva columna para tooltip con fecha en español
source['Fecha_str'] = source['Fecha'].dt.day.astype(str) + ' de ' + \
                     source['Fecha'].dt.month.map(meses_es) + ' de ' + \
                     source['Fecha'].dt.year.astype(str)

# Mantener solo un concierto si tiene misma fecha y mismo recinto
# (conciertos con la misma fecha pero en recintos distintos se conservan)
source = source.drop_duplicates(subset=['Fecha', 'Recinto'])

# Mostrar cantidad final de conciertos únicos que se grafican
print(f"Total de conciertos: {len(source)}")

# Definir paleta arcoíris inversa (de azul a rojo)
palette_invertida = [
    '#0000FF', '#0055FF', '#00AAFF', '#00FFAA', '#AAFF00',
    '#FFFF00', '#FFAA00', '#FF5500', '#FF0000', '#FF007F',
    '#FF00AA', '#AA00FF', '#5500FF'
]

# Crear gráfico
chart = alt.Chart(source).mark_circle(size=60).encode(
    x=alt.X('Fecha:T',
            title='Días del año',
            axis=alt.Axis(format='%d %b', labelAngle=-45),
            scale=alt.Scale(domain=[source['Fecha'].min(), source['Fecha'].max()])),
    y=alt.Y('y_random:Q', axis=None),
    color=alt.Color('Recinto:N',
                    scale=alt.Scale(range=palette_invertida),
                    legend=alt.Legend(title="Recinto")),
    tooltip=[
        'Concierto',
        'Artista',
        'Recinto',
        alt.Tooltip('Fecha_str:N', title='Fecha')
    ]
).properties(
    title='Conciertos masivos (de más de 3000 asistentes) realizados en 2024',
    width=1000,
    height=300
).interactive()

chart.show()


Total de conciertos: 197


alt.Chart(...)

In [ ]:
import numpy as np
import pandas as pd
from urllib import request
import json
import altair as alt

# Leer datos
source = pd.read_csv('/content/drive/MyDrive/concierto_database_limpia_2.csv', delimiter=';')
source['Fecha'] = pd.to_datetime(source['Fecha'], format='%d-%m-%Y', errors='coerce')
source = source[source['Cancelado'] != 1]
source = source.drop_duplicates(subset=['Fecha', 'Recinto'])

# Crear todas las fechas del año
fechas_2024 = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
df_dias = pd.DataFrame({'Fecha': fechas_2024})

# Merge con datos originales
source_full = pd.merge(df_dias, source, on='Fecha', how='left')

# y_random solo si hay concierto
np.random.seed(42)
source_full['y_random'] = np.where(source_full['Concierto'].notna(),
                                   np.random.uniform(0, 1, size=len(source_full)),
                                   np.nan)

# Fecha legible
meses_es = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
    5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
    9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}
source_full['Fecha_str'] = source_full['Fecha'].dt.day.astype(str) + ' de ' + \
                           source_full['Fecha'].dt.month.map(meses_es) + ' de ' + \
                           source_full['Fecha'].dt.year.astype(str)

# Localización español
with request.urlopen('https://raw.githubusercontent.com/d3/d3-format/master/locale/es-MX.json') as f:
    de_format = json.load(f)
with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-MX.json') as f:
    de_time_format = json.load(f)
alt.renderers.set_embed_options(formatLocale=de_format, timeFormatLocale=de_time_format)

# Paleta colores
palette_invertida = [
    '#0000FF', '#0055FF', '#00AAFF', '#00FFAA', '#AAFF00',
    '#FFFF00', '#FFAA00', '#FF5500', '#FF0000', '#FF007F',
    '#FF00AA', '#AA00FF', '#5500FF'
]

# CREAR GRÁFICO CON TODOS LOS DÍAS EN EL EJE X
chart = alt.Chart(source_full.dropna(subset=['y_random'])).mark_circle(size=60).encode(
    x=alt.X('Fecha:T',
            title='Día del año',
            axis=alt.Axis(
                values=fechas_2024,  # ← mostrar los 365 días
                format='%d %b',
                labelAngle=-90,
                labelFontSize=8,
                tickCount=365
            ),
            scale=alt.Scale(domain=[pd.to_datetime('2024-01-01'), pd.to_datetime('2024-12-31')])
    ),
    y=alt.Y('y_random:Q', axis=None),
    color=alt.Color('Recinto:N',
                    scale=alt.Scale(range=palette_invertida),
                    legend=alt.Legend(title="Recinto")),
    tooltip=[
        'Concierto',
        'Artista',
        'Recinto',
        alt.Tooltip('Fecha_str:N', title='Fecha')
    ]
).properties(
    title='Conciertos masivos en la Región Metropolitana durante 2024',
    width=3000,
    height=200
).interactive()

chart.show()

alt.Chart(...)

In [ ]:
chart.save('vis_03.html')

In [ ]:
from google.colab import files
files.download('vis_03.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install vl-convert-python

import altair as alt
alt.data_transformers.disable_max_rows()

# Guardar como PNG
chart.save('vis_03.png')

from PIL import Image

img = Image.open('vis_03.png')
rgb_img = img.convert('RGB')
rgb_img.save('vis_03.jpg')

from google.colab import files
files.download('vis_03.jpg')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 47.6 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import pandas as pd
from urllib import request
import json
import altair as alt

# Leer datos desde Drive
source = pd.read_csv('/content/drive/MyDrive/concierto_database_limpia_2.csv', delimiter=';')

# Asegurar formato de fecha
source['Fecha'] = pd.to_datetime(source['Fecha'], format='%d-%m-%Y', errors='coerce')
source['Mes'] = source['Fecha'].dt.month

# Filtrar: solo conciertos NO cancelados
source = source[source['Cancelado'] != 1]

# Filtrar: solo Movistar Arena
source = source[source['Recinto'].str.lower().str.contains("movistar arena")]

# Añadir Y aleatorio para separar puntos
np.random.seed(42)
source['y_random'] = np.random.uniform(0, 1, size=len(source))

# Diccionario meses en español
meses_es = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
    5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
    9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}

# Formato español para fechas en tooltip
source['Fecha_str'] = source['Fecha'].dt.day.astype(str) + ' de ' + \
                      source['Fecha'].dt.month.map(meses_es) + ' de ' + \
                      source['Fecha'].dt.year.astype(str)

# Eliminar duplicados: misma fecha y recinto
source = source.drop_duplicates(subset=['Fecha', 'Recinto'])

# Descargar y aplicar configuración local en español
with request.urlopen('https://raw.githubusercontent.com/d3/d3-format/master/locale/es-MX.json') as f:
  de_format = json.load(f)

with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-MX.json') as f:
  de_time_format = json.load(f)

alt.renderers.set_embed_options(formatLocale=de_format, timeFormatLocale=de_time_format)

# Mostrar cantidad de conciertos únicos en Movistar Arena
print(f"Total de conciertos en Movistar Arena: {len(source)}")

# Paleta arcoíris invertida
palette_invertida = [
    '#0000FF', '#0055FF', '#00AAFF', '#00FFAA', '#AAFF00',
    '#FFFF00', '#FFAA00', '#FF5500', '#FF0000', '#FF007F',
    '#FF00AA', '#AA00FF', '#5500FF'
]

# Crear gráfico
chart = alt.Chart(source).mark_circle(size=60).encode(
    x=alt.X('Fecha:T',
            title='Días del año',
            axis=alt.Axis(format='%d %b', labelAngle=-45),
            scale=alt.Scale(domain=[source['Fecha'].min(), source['Fecha'].max()])),
    y=alt.Y('y_random:Q', axis=None),
    color=alt.Color('Recinto:N',
                    scale=alt.Scale(range=palette_invertida),
                    legend=alt.Legend(title="Recinto")),
    tooltip=[
        'Concierto',
        'Artista',
        'Recinto',
        alt.Tooltip('Fecha_str:N', title='Fecha')
    ]
).properties(
    title='Conciertos realizados en el Movistar Arena (2024)',
    width=1000,
    height=300
).interactive()

chart.show()


Total de conciertos en Movistar Arena: 114


alt.Chart(...)

In [ ]:
import numpy as np
import pandas as pd
from urllib import request
import json
import altair as alt

# Leer datos
source = pd.read_csv('/content/drive/MyDrive/concierto_database_limpia_2.csv', delimiter=';')
source['Fecha'] = pd.to_datetime(source['Fecha'], format='%d-%m-%Y', errors='coerce')

# Filtrar solo Movistar Arena en 2024
source = source[
    (source['Cancelado'] != 1) &
    (source['Recinto'].str.lower().str.contains('movistar arena')) &
    (source['Fecha'].dt.year == 2024)
]

# Agrupar por día: contar conciertos
conteo_diario = source.groupby('Fecha').size().reset_index(name='Conciertos')

# Cargar formatos en español
with request.urlopen('https://raw.githubusercontent.com/d3/d3-format/master/locale/es-MX.json') as f:
  de_format = json.load(f)
with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-MX.json') as f:
  de_time_format = json.load(f)

alt.renderers.set_embed_options(formatLocale=de_format, timeFormatLocale=de_time_format)

# Si hay pocos datos, loess no es tan útil. Alternativa: usar rolling mean (media móvil)
# Vamos a hacerlo nosotros con pandas
conteo_diario = conteo_diario.set_index('Fecha').asfreq('D', fill_value=0)  # asegurar todos los días del año
conteo_diario['Rolling'] = conteo_diario['Conciertos'].rolling(window=7, center=True, min_periods=1).mean()
conteo_diario = conteo_diario.reset_index()

# Gráfico con puntos + línea suavizada (rolling mean)
base = alt.Chart(conteo_diario).encode(
    x=alt.X('Fecha:T', title='Fecha', axis=alt.Axis(format='%b', labelAngle=-45))
)

linea = base.mark_line(color='#A20264').encode(
    y=alt.Y('Rolling:Q', title='Conciertos (media móvil 7 días)')
)

puntos = base.mark_circle(opacity=0.4, size=30).encode(
    y='Conciertos:Q',
    tooltip=[
        alt.Tooltip('Fecha:T', title='Fecha'),
        alt.Tooltip('Conciertos:Q', title='Conciertos ese día')
    ]
)

chart = (puntos + linea).properties(
    title='Evolución diaria de conciertos en el Movistar Arena (2024)',
    width=900,
    height=300
)

chart.show()



alt.LayerChart(...)

In [ ]:
import numpy as np
import pandas as pd
from urllib import request
import json
import altair as alt

# Leer y preparar datos
source = pd.read_csv('/content/drive/MyDrive/concierto_database_limpia_2.csv', delimiter=';')
source['Fecha'] = pd.to_datetime(source['Fecha'], format='%d-%m-%Y', errors='coerce')
# Mantener solo un concierto si tiene misma fecha y mismo recinto
# (conciertos con la misma fecha pero en recintos distintos se conservan)
source = source.drop_duplicates(subset=['Fecha', 'Recinto'])
# Filtrar conciertos en Movistar Arena durante 2024
source = source[
    (source['Cancelado'] != 1) &
    (source['Recinto'].str.lower().str.contains('movistar arena')) &
    (source['Fecha'].dt.year == 2024)
]

# Agrupar por mes
source['Mes_num'] = source['Fecha'].dt.month
meses_es = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
    5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
    9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}
source['Mes'] = source['Mes_num'].map(meses_es)

conteo_mensual = source.groupby(['Mes_num', 'Mes']).size().reset_index(name='Conciertos')
conteo_mensual = conteo_mensual.sort_values('Mes_num')

# Formato español para Altair
with request.urlopen('https://raw.githubusercontent.com/d3/d3-format/master/locale/es-MX.json') as f:
    de_format = json.load(f)
with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-MX.json') as f:
    de_time_format = json.load(f)
alt.renderers.set_embed_options(formatLocale=de_format, timeFormatLocale=de_time_format)

# Gráfico línea con puntos, ambos del mismo color
chart = alt.Chart(conteo_mensual).mark_line(point=True, color='#49007E').encode(
    x=alt.X('Mes:N', sort=list(meses_es.values()), title='Mes'),
    y=alt.Y('Conciertos:Q', title='Cantidad de conciertos'),
    tooltip=['Mes', 'Conciertos']
).properties(
    title='Conciertos mensuales en el Movistar Arena (2024)',
    width=700,
    height=350
)

chart.show()


alt.Chart(...)

In [ ]:
import numpy as np
import pandas as pd
from urllib import request
import json
import altair as alt

# Leer y preparar datos
source = pd.read_csv('/content/drive/MyDrive/concierto_database_limpia_2.csv', delimiter=';')
source['Fecha'] = pd.to_datetime(source['Fecha'], format='%d-%m-%Y', errors='coerce')
source = source.drop_duplicates(subset=['Fecha', 'Recinto'])

# Filtrar conciertos en Movistar Arena durante 2024
source = source[
    (source['Cancelado'] != 1) &
    (source['Recinto'].str.lower().str.contains('movistar arena')) &
    (source['Fecha'].dt.year == 2024)
]

# Crear columnas de día y mes
source['Día'] = source['Fecha'].dt.day
source['Mes_num'] = source['Fecha'].dt.month
meses_es = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
    5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
    9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}
source['Mes'] = source['Mes_num'].map(meses_es)

# Agrupar por día y mes
conteo_diario = source.groupby(['Fecha', 'Día', 'Mes_num', 'Mes']).size().reset_index(name='Conciertos')

# Formato en español para Altair
with request.urlopen('https://raw.githubusercontent.com/d3/d3-format/master/locale/es-MX.json') as f:
    de_format = json.load(f)
with request.urlopen('https://raw.githubusercontent.com/d3/d3-time-format/master/locale/es-MX.json') as f:
    de_time_format = json.load(f)
alt.renderers.set_embed_options(formatLocale=de_format, timeFormatLocale=de_time_format)

# Crear heatmap diario tipo calendario
calendar = alt.Chart(conteo_diario).mark_rect().encode(
    x=alt.X('Día:O', title='Día del mes', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Mes:N', sort=list(meses_es.values()), title='Mes'),
color=alt.Color('Conciertos:Q',
    scale=alt.Scale(
        domain=[1, conteo_diario['Conciertos'].max()],
        range=['#ffe6f0', '#ff99b5', '#ff4c84', '#ff005b', '#cc0049', '#80002f']
    ), legend=alt.Legend(title='Conciertos')),
    tooltip=[
        alt.Tooltip('Fecha:T', title='Fecha'),
        alt.Tooltip('Conciertos:Q', title='Conciertos')
    ]
).properties(
    title='Mapa de calor diario: Conciertos realizados en el Movistar Arena en 2024',
    width=700,
    height=350
).configure_view(
    stroke=None
)

calendar.show()


alt.Chart(...)